In [ ]:

# SECTION 1: ENVIRONMENT SETUP

# Install git-lfs for downloading large model files
!apt-get -y -qq install git-lfs > /dev/null 2>&1

# Remove any existing installation to ensure clean state
!rm -rf mattersim_repo

# Clone MatterSim repo with a distinct name to avoid import conflicts
!git clone https://github.com/microsoft/mattersim.git mattersim_repo

%cd /content/mattersim_repo

# Initialize git-lfs and download pretrained model weights
!git lfs install
!git lfs pull -I "pretrained_models/*" --exclude=""

# Install the package in editable mode along with required dependencies
# -q flag suppresses verbose output
!pip install -q -e .
!pip install -q ase tqdm

print("✓ Environment setup complete")

Cloning into 'mattersim_repo'...
remote: Enumerating objects: 1515, done.
remote: Counting objects: 100% (317/317), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 1515 (delta 243), reused 165 (delta 165), pack-reused 1198 (from 2)
Receiving objects: 100% (1515/1515), 132.83 MiB | 16.92 MiB/s, done.
Resolving deltas: 100% (729/729), done.
/content/mattersim_repo
Updated git hooks.
Git LFS initialized.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mattersim (pyproject.toml) ... done
✓ Environment setup complete


In [ ]:
# SECTION 2: VERIFY INSTALLATION AND IMPORTS

import sys
from pathlib import Path
import torch

# Define paths
REPO_ROOT = Path("/content/mattersim_repo")
SRC_DIR = REPO_ROOT / "src"

# Verify repo exists
if not REPO_ROOT.exists():
    raise RuntimeError(
        f"Repository not found at {REPO_ROOT}. "
        "Please run Section 1 first."
    )

# Add source directory to Python path to ensure imports work
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# Import MatterSim components
try:
    import mattersim
    from mattersim.forcefield import MatterSimCalculator
    print(f"✓ MatterSim imported from: {mattersim.__file__}")
except ImportError as e:
    raise ImportError(
        "Failed to import mattersim. Check that Section 1 completed successfully."
    ) from e

# Import other required packages
from ase.io import read, write
from ase.optimize import FIRE
import pandas as pd
from tqdm import tqdm

# Verify pretrained models exist
PRETRAINED_DIR = REPO_ROOT / "pretrained_models"
model_files = sorted(PRETRAINED_DIR.glob("*.pth"))

if not model_files:
    raise RuntimeError(
        f"No pretrained model files found in {PRETRAINED_DIR}. "
        "Check that git-lfs pull succeeded."
    )

print("\nAvailable models:")
for model in model_files:
    print(f"  - {model.name}")

# Select model (prefer 1M for speed, fall back to first available)
MODEL_PATH = None
for model in model_files:
    if "1M" in model.name:
        MODEL_PATH = model
        break
if MODEL_PATH is None:
    MODEL_PATH = model_files[0]

print(f"\n✓ Using model: {MODEL_PATH.name}")

# Check GPU availability
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✓ Computing device: {DEVICE}")
if DEVICE == "cpu":
    print("  ⚠ Warning: GPU not detected. Calculations will be slow.")
    print("  Consider: Runtime > Change runtime type > Hardware accelerator > GPU")

✓ MatterSim imported from: /content/mattersim_repo/src/mattersim/__init__.py

Available models:
  - mattersim-v1.0.0-1M.pth
  - mattersim-v1.0.0-5M.pth

✓ Using model: mattersim-v1.0.0-1M.pth
✓ Computing device: cuda


In [ ]:
# SECTION 3: UPLOAD AND EXTRACT STRUCTURES

from google.colab import files
uploaded = files.upload()

# Extract Files
!unzip combined_cif.zip -d combined_cif
!ls combined_cif | head

Saving combined_cif.zip to combined_cif (2).zip
Archive:  combined_cif.zip
replace combined_cif/__MACOSX/._Master_CIF_Consolidation? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: combined_cif/__MACOSX/._Master_CIF_Consolidation  
  inflating: combined_cif/Master_CIF_Consolidation/struct_0540.cif  
  inflating: combined_cif/__MACOSX/Master_CIF_Consolidation/._struct_0540.cif  
  inflating: combined_cif/Master_CIF_Consolidation/struct_0226.cif  
  inflating: combined_cif/__MACOSX/Master_CIF_Consolidation/._struct_0226.cif  
  inflating: combined_cif/Master_CIF_Consolidation/struct_0232.cif  
  inflating: combined_cif/__MACOSX/Master_CIF_Consolidation/._struct_0232.cif  
  inflating: combined_cif/Master_CIF_Consolidation/struct_0554.cif  
  inflating: combined_cif/__MACOSX/Master_CIF_Consolidation/._struct_0554.cif  
  inflating: combined_cif/Master_CIF_Consolidation/struct_0568.cif  
  inflating: combined_cif/__MACOSX/Master_CIF_Consolidation/._struct_0568.cif  
  inflating: combi

In [ ]:
# Define CIF directory path
# Adjust this if your zip has a different internal structure
CIF_DIR = Path("/content/mattersim_repo/combined_cif/Master_CIF_Consolidation")

if not CIF_DIR.exists():
    raise RuntimeError(
        f"Expected CIF directory at {CIF_DIR} but it doesn't exist. "
        f"Check the internal structure of your zip file."
    )

# Count and display sample files
cif_files = sorted(CIF_DIR.glob("*.cif"))
print(f"\n✓ Found {len(cif_files)} CIF files")
print("\nFirst 5 files:")
for cif in cif_files[:5]:
    print(f"  - {cif.name}")


✓ Found 600 CIF files

First 5 files:
  - struct_0001.cif
  - struct_0002.cif
  - struct_0003.cif
  - struct_0004.cif
  - struct_0005.cif


In [ ]:
# SECTION 4: INITIALIZE CALCULATOR

print("Initializing MatterSim calculator...")
print(f"  Model: {MODEL_PATH.name}")
print(f"  Device: {DEVICE}")

calc = MatterSimCalculator(
    load_path=str(MODEL_PATH),
    device=DEVICE
)

print("✓ Calculator initialized")

Initializing MatterSim calculator...
  Model: mattersim-v1.0.0-1M.pth
  Device: cuda
2025-12-11 23:23:19.350 | INFO     | mattersim.forcefield.potential:from_checkpoint:893 - Loading the model from /content/mattersim_repo/pretrained_models/mattersim-v1.0.0-1M.pth
✓ Calculator initialized


In [ ]:
# SECTION 5: SINGLE-STRUCTURE TEST

print("Testing pipeline with struct_0001.cif...")

test_cif = CIF_DIR / "struct_0001.cif"
if not test_cif.exists():
    raise RuntimeError(f"Test file {test_cif} not found")

# Read structure
atoms = read(str(test_cif))
print(f"  Atoms: {len(atoms)}")
print(f"  Formula: {atoms.get_chemical_formula()}")

# Attach calculator and compute energy
atoms.calc = calc
energy = atoms.get_potential_energy()
forces = atoms.get_forces()

print(f"  Energy: {energy:.6f} eV")
print(f"  Forces shape: {forces.shape}")
print("✓ Single-structure test passed")

Testing pipeline with struct_0001.cif...
  Atoms: 10
  Formula: Ca2O6SiTi
  Energy: -76.974426 eV
  Forces shape: (10, 3)
✓ Single-structure test passed


In [ ]:
# SECTION 6: COMPUTE SINGLE-POINT ENERGIES

# Create output directory
ENERGY_OUT_DIR = Path("/content/mattersim_results")
ENERGY_OUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Computing single-point energies for {len(cif_files)} structures...")

energy_records = []

for cif_file in tqdm(cif_files, desc="Energy evaluation"):
    try:
        # Read structure
        atoms = read(str(cif_file))

        # Attach calculator
        atoms.calc = calc

        # Compute energy
        energy = atoms.get_potential_energy()

        # Store results
        energy_records.append({
            "file": cif_file.name,
            "energy_eV": energy,
            "energy_per_atom_eV": energy / len(atoms),
            "natoms": len(atoms),
            "formula": atoms.get_chemical_formula(),
        })

    except Exception as e:
        print(f"  ✗ Failed: {cif_file.name} - {e}")
        energy_records.append({
            "file": cif_file.name,
            "error": str(e),
        })

# Save results
energy_df = pd.DataFrame(energy_records)
energy_csv = ENERGY_OUT_DIR / "energy_summary.csv"
energy_df.to_csv(energy_csv, index=False)

print(f"\n✓ Results saved to: {energy_csv}")
print(f"✓ Successfully processed: {energy_df['energy_eV'].notna().sum()}/{len(cif_files)}")

# Display summary statistics
if not energy_df.empty and 'energy_per_atom_eV' in energy_df.columns:
    valid_energies = energy_df['energy_per_atom_eV'].dropna()
    if len(valid_energies) > 0:
        print("\nEnergy statistics (eV/atom):")
        print(f"  Mean:   {valid_energies.mean():.4f}")
        print(f"  Median: {valid_energies.median():.4f}")
        print(f"  Std:    {valid_energies.std():.4f}")
        print(f"  Range:  [{valid_energies.min():.4f}, {valid_energies.max():.4f}]")

energy_df.head(10)

Computing single-point energies for 600 structures...


Energy evaluation: 100%|██████████| 600/600 [00:26<00:00, 22.64it/s]


✓ Results saved to: /content/mattersim_results/energy_summary.csv
✓ Successfully processed: 600/600

Energy statistics (eV/atom):
  Mean:   -7.0664
  Median: -7.3772
  Std:    0.7821
  Range:  [-8.6754, -3.9018]


,file,energy_eV,energy_per_atom_eV,natoms,formula
0,struct_0001.cif,-76.974426,-7.697443,10,Ca2O6SiTi
1,struct_0002.cif,-155.168228,-7.758411,20,Ca4O12Si2Ti2
2,struct_0003.cif,-40.979244,-6.829874,6,CaOSi2Ti2
3,struct_0004.cif,-89.763947,-7.480329,12,Ca2O7Si2Ti
4,struct_0005.cif,-89.920410,-7.493368,12,CaO8Si2Ti
5,struct_0006.cif,-73.742264,-6.703842,11,Ca2O2Si3Ti4
6,struct_0007.cif,-151.654800,-7.582740,20,Ca2O12Si4Ti2
7,struct_0008.cif,-85.161270,-7.096773,12,Ca2O6Si3Ti
8,struct_0009.cif,-25.808531,-6.452133,4,CaOSiTi
9,struct_0010.cif,-25.437866,-6.359467,4,CaOSiTi


In [ ]:
# SECTION 7: GEOMETRY RELAXATION

# Create output directory
RELAX_OUT_DIR = Path("/content/mattersim_results_relaxed")
RELAX_OUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Relaxing {len(cif_files)} structures...")
print("  Convergence criterion: fmax = 0.02 eV/Å")
print("  Maximum steps: 200\n")

relax_records = []

for cif_file in tqdm(cif_files, desc="Relaxation"):
    try:
        # Read structure
        atoms = read(str(cif_file))
        atoms.calc = calc

        # Energy before relaxation
        e_before = atoms.get_potential_energy()

        # Run FIRE optimization
        # logfile=None suppresses per-step output
        optimizer = FIRE(atoms, logfile=None)
        optimizer.run(fmax=0.02, steps=200)

        # Energy after relaxation
        e_after = atoms.get_potential_energy()

        # Save relaxed structure
        relaxed_name = cif_file.stem + "_relaxed.cif"
        relaxed_path = RELAX_OUT_DIR / relaxed_name
        write(str(relaxed_path), atoms)

        # Record results
        relax_records.append({
            "file": cif_file.name,
            "energy_before_eV": e_before,
            "energy_after_eV": e_after,
            "delta_energy_eV": e_after - e_before,
            "energy_per_atom_eV": e_after / len(atoms),
            "natoms": len(atoms),
            "formula": atoms.get_chemical_formula(),
            "relaxed_file": relaxed_name,
        })

    except Exception as e:
        print(f"  ✗ Failed: {cif_file.name} - {e}")
        relax_records.append({
            "file": cif_file.name,
            "error": str(e),
        })

# Save results
relax_df = pd.DataFrame(relax_records)
relax_csv = RELAX_OUT_DIR / "relaxation_summary.csv"
relax_df.to_csv(relax_csv, index=False)

print(f"\n✓ Results saved to: {relax_csv}")
print(f"✓ Successfully relaxed: {relax_df['energy_after_eV'].notna().sum()}/{len(cif_files)}")

# Display summary statistics
if not relax_df.empty and 'delta_energy_eV' in relax_df.columns:
    valid_deltas = relax_df['delta_energy_eV'].dropna()
    if len(valid_deltas) > 0:
        print("\nRelaxation energy change (eV):")
        print(f"  Mean:   {valid_deltas.mean():.4f}")
        print(f"  Median: {valid_deltas.median():.4f}")
        print(f"  Std:    {valid_deltas.std():.4f}")
        print(f"  Range:  [{valid_deltas.min():.4f}, {valid_deltas.max():.4f}]")

relax_df.head(10)

Relaxing 600 structures...
  Convergence criterion: fmax = 0.02 eV/Å
  Maximum steps: 200



Relaxation: 100%|██████████| 600/600 [14:32<00:00,  1.45s/it]


✓ Results saved to: /content/mattersim_results_relaxed/relaxation_summary.csv
✓ Successfully relaxed: 600/600

Relaxation energy change (eV):
  Mean:   -0.2242
  Median: -0.0489
  Std:    0.5815
  Range:  [-6.9310, 0.0000]


,file,energy_before_eV,energy_after_eV,delta_energy_eV,energy_per_atom_eV,natoms,formula,relaxed_file
0,struct_0001.cif,-76.974426,-76.974709,-0.000282,-7.697471,10,Ca2O6SiTi,struct_0001_relaxed.cif
1,struct_0002.cif,-155.168228,-155.221130,-0.052902,-7.761056,20,Ca4O12Si2Ti2,struct_0002_relaxed.cif
2,struct_0003.cif,-40.979244,-40.981831,-0.002586,-6.830305,6,CaOSi2Ti2,struct_0003_relaxed.cif
3,struct_0004.cif,-89.763939,-90.015366,-0.251427,-7.501280,12,Ca2O7Si2Ti,struct_0004_relaxed.cif
4,struct_0005.cif,-89.920410,-89.978416,-0.058006,-7.498201,12,CaO8Si2Ti,struct_0005_relaxed.cif
5,struct_0006.cif,-73.742264,-74.051117,-0.308853,-6.731920,11,Ca2O2Si3Ti4,struct_0006_relaxed.cif
6,struct_0007.cif,-151.654785,-152.220367,-0.565582,-7.611018,20,Ca2O12Si4Ti2,struct_0007_relaxed.cif
7,struct_0008.cif,-85.161255,-85.277344,-0.116089,-7.106445,12,Ca2O6Si3Ti,struct_0008_relaxed.cif
8,struct_0009.cif,-25.808531,-25.834793,-0.026262,-6.458698,4,CaOSiTi,struct_0009_relaxed.cif
9,struct_0010.cif,-25.437866,-25.470547,-0.032681,-6.367637,4,CaOSiTi,struct_0010_relaxed.cif


In [ ]:
# SECTION 8: PACKAGE RESULTS

from google.colab import files

print("Creating zip archives...")

# Zip relaxed structures and summary
!zip -r -q /content/mattersim_results_relaxed.zip /content/mattersim_results_relaxed

# Zip both CSV summaries
!zip -j -q /content/mattersim_csv_summaries.zip \
    /content/mattersim_results/energy_summary.csv \
    /content/mattersim_results_relaxed/relaxation_summary.csv

print("✓ Archives created")
print("\nDownloading files...")

# Download relaxed structures
print("  1. Downloading mattersim_results_relaxed.zip...")
files.download("/content/mattersim_results_relaxed.zip")

# Download CSV summaries
print("  2. Downloading mattersim_csv_summaries.zip...")
files.download("/content/mattersim_csv_summaries.zip")

print("\n✓ Downloads complete")

Creating zip archives...
✓ Archives created

  1. Downloading mattersim_results_relaxed.zip...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  2. Downloading mattersim_csv_summaries.zip...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✓ Downloads complete
